In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *

%load_ext autoreload
%autoreload 2

In [4]:
def least_squares(y, tx):
    (a, b) = tx.T @ tx, tx.T @ y
    ws = np.linalg.lstsq(a, b,rcond=None)
    w = ws[0]
    # Calculate residuals
    res = y - tx @ w
    # Calculate the Mean Squared Error
    mse = (res.T @ res)/(2*len(y))
    return w, mse

test_y = np.array([0.1, 0.3, 0.5])
test_x = np.array([[2.3, 3.2], [1.0, 0.1], [1.4, 2.3]])
test_res_w, test_res_mse = least_squares(test_y, test_x)
print(test_res_w, test_res_mse)
np.testing.assert_allclose(test_res_w, np.array([0.218786, -0.053837]), rtol=1e-4, atol=1e-8)
np.testing.assert_allclose(test_res_mse, 0.026942, rtol=1e-4, atol=1e-8)

[ 0.2187858  -0.05383734] 0.026941580756013744


In [9]:
# load dataset
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data/dataset_to_release")

In [10]:
mean = np.nanmean(x_train, axis=0)

# Loop through each column
for col in range(x_train.shape[1]):
    nan_mask = np.isnan(x_train[:, col])
    x_train[nan_mask, col] = mean[col]

x_train.shape, y_train.shape, x_train[0,:], y_train[2]

((328135, 321),
 (328135,),
 array([5.30000000e+01, 1.10000000e+01, 1.11620150e+07, 1.10000000e+01,
        1.60000000e+01, 2.01500000e+03, 1.10000000e+03, 2.01501563e+09,
        2.01501563e+09, 1.00000000e+00, 1.00016956e+00, 1.00000000e+00,
        1.00000000e+00, 1.54463226e+00, 1.65625000e+00, 1.79388666e+00,
        8.01570428e-01, 9.91935740e-01, 1.00000000e+00, 1.00000000e+00,
        2.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 2.00000000e+00, 2.00000000e+00, 1.00000000e+00,
        5.00000000e+00, 8.80000000e+01, 1.00000000e+00, 1.00000000e+00,
        2.00000000e+00, 1.00000000e+00, 3.00000000e+00, 1.17256318e+00,
        1.00000000e+00, 1.00000000e+00, 2.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 1.48319767e+00, 2.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 2.00000000e+00, 1.00000000e+00, 2.00000000e+00,
        3.00000000e+00, 5.44746802e+01, 2.00000000e+00, 1.00000000e+00,
        5.00000000e+00, 1.00000000e+

In [11]:
def standardize(x):
    """
    Classical Standardisation function
    
    Args:
        x: Data in shape (N, D)
        
    Returns:
        x: As standardised data
        mean: Mean of each column
        std: Standard deviation for each column
    """
    mean = np.mean(x)
    x = x - mean
    std = np.std(x)
    x = x/std
    return x, mean, std

In [12]:
x_train, mean_x, std_x = standardize(x_train)
x_train[3, :]

array([-0.07932676, -0.07932699, -0.04033713, -0.07932699, -0.07932691,
       -0.07931432, -0.07932009, 12.6292844 , 12.6292844 , -0.07932702,
       -0.07932702, -0.07932702, -0.07932702, -0.07932701, -0.07932701,
       -0.07932701, -0.07932702, -0.07932702, -0.07932702, -0.07932702,
       -0.07932701, -0.07932702, -0.07932702, -0.07932702, -0.07932701,
       -0.07932701, -0.07932702, -0.07932647, -0.07932647, -0.07932667,
       -0.07932702, -0.07932702, -0.07932701, -0.07932702, -0.07932701,
       -0.07932702, -0.07932702, -0.079327  , -0.07932701, -0.07932701,
       -0.07932701, -0.07932702, -0.07932701, -0.07932701, -0.07932701,
       -0.07932702, -0.07932701, -0.07932701, -0.07932701, -0.07932668,
       -0.07932701, -0.07932702, -0.079327  , -0.07932702, -0.07932701,
       -0.07932701, -0.07932702, -0.07932701, -0.07932698, -0.07932647,
       -0.0793264 , -0.07932701, -0.07932597, -0.07932384, -0.07932701,
       -0.07932701, -0.07932701, -0.07932701, -0.07932701, -0.07

In [13]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree.

    Args:
        x: numpy array of shape (N,), N is the number of samples.
        degree: integer.

    Returns:
        poly: numpy array of shape (N,d+1)

    >>> build_poly(np.array([0.0, 1.5]), 2)
    array([[1.  , 0.  , 0.  ],
           [1.  , 1.5 , 2.25]])
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # polynomial basis function: TODO
    # this function should return the matrix formed
    # by applying the polynomial basis to the input data
    # ***************************************************
    N, D = x.shape
    poly = np.zeros((N, D * (degree + 1)))

    for j in range(degree + 1):
        poly[:, j * D:(j + 1) * D] = x ** j

    return poly


In [14]:
def build_model_data(x, y, degree=1):
    poly = build_poly(x, degree)
    return least_squares(y, poly)

In [ ]:
mes = []
for i in range(10):
    w, mse = build_model_data(x_train, y_train, i)
    print("Degree: ", i, "MSE: ", mse)
    mes.append(mse)
    
plt.plot(mes)

Degree:  0 MSE:  0.16100963017504696
Degree:  1 MSE:  0.1603791082788717
Degree:  2 MSE:  0.1609113576776
Degree:  3 MSE:  0.1610047203334176
Degree:  4 MSE:  0.1610059985398393
Degree:  5 MSE:  0.16100938797939346
Degree:  6 MSE:  0.1610093394873732
Degree:  7 MSE:  0.1610092909981258
Degree:  8 MSE:  0.16100924251165089


In [ ]:
y_class = np.expand_dims(y_train, axis=1)
y_class[y_class == -1] = 0
y_class

# Logistic Regression using gradient descent or SGD

In [ ]:
def sigmoid(t):
    """apply sigmoid function on t.

    Args:
        t: scalar or numpy array

    Returns:
        scalar or numpy array

    >>> sigmoid(np.array([0.1]))
    array([0.52497919])
    >>> sigmoid(np.array([0.1, 0.1]))
    array([0.52497919, 0.52497919])
    """
    return 1 / (1 + np.exp(-t))

In [ ]:
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a non-negative loss

    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(4).reshape(2, 2)
    >>> w = np.c_[[2., 3.]]
    >>> round(calculate_loss(y, tx, w), 8)
    1.52429481
    """
    assert y.shape[0] == tx.shape[0]
    assert tx.shape[1] == w.shape[0]

    N = y.shape[0]
    loss = (-1 / N) * np.sum(y*np.log(sigmoid(tx @ w)) + (1-y)*np.log(1-sigmoid(tx @ w)))
    return loss

In [ ]:
import random
def calculate_gradient_sgd(y, tx, w):
    """compute the gradient of loss.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a vector of shape (D, 1)
    """
    index = random.randint(0, y.shape[0] - 1)
    xn = tx[index, :]
    yn = y[index, :]
    return xn * (sigmoid(xn @ w) - yn)

def calculate_gradient(y, tx, w):
    """compute the gradient of loss.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a vector of shape (D, 1)

    >>> np.set_printoptions(8)
    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(6).reshape(2, 3)
    >>> w = np.array([[0.1], [0.2], [0.3]])
    >>> calculate_gradient(y, tx, w)
    array([[-0.10370763],
           [ 0.2067104 ],
           [ 0.51712843]])
    """
    N = y.shape[0]
    return (1 / N) * tx.T @ (sigmoid(tx @ w)-y)

def calculate_gradient_mb(y, tx, w, batch_size=30):
    batch_indices = random.sample(range(y.shape[0]), batch_size)
    X = tx[batch_indices]
    Y = y[batch_indices]
    N = len(batch_indices)
    return (1/N) * X.T @ (sigmoid(X @ w)-Y)

In [ ]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression. Return the loss and the updated w.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)
        gamma: float

    Returns:
        loss: scalar number
        w: shape=(D, 1)

    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(6).reshape(2, 3)
    >>> w = np.array([[0.1], [0.2], [0.3]])
    >>> gamma = 0.1
    >>> loss, w = learning_by_gradient_descent(y, tx, w, gamma)
    >>> round(loss, 8)
    0.62137268
    >>> w
    array([[0.11037076],
           [0.17932896],
           [0.24828716]])
    """
    loss = calculate_loss(y, tx, w)
    gradient = calculate_gradient(y, tx, w)
    return loss, w - gamma * gradient

In [ ]:
def logistic_regression(y, tx, initial_w=None, max_iter=10000, gamma=0.1, threshold=1e-12):
    """return the loss, gradient of the loss

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        initial_w:  shape=(D, 1)
        max_iter: int and maximum amount of iterations
        gamma: float and learning rate
        threshold: float and threshold as absolute diff to stop

    Returns:
        loss: scalar number
        gradient: shape=(D, 1)

    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(6).reshape(2, 3)
    >>> w = np.array([[0.1], [0.2], [0.3]])
    >>> gradient, loss = logistic_regression(y, tx, w)
    >>> round(loss, 8)
    0.62137268
    >>> gradient
    (array([[-0.10370763],
           [ 0.2067104 ],
           [ 0.51712843]]))
    """
    losses = []
    w = np.zeros((tx.shape[1], 1)) if initial_w is None else initial_w
    # Iterate over max iterations. Stop when we get convergence under threshold.
    for iter in range(max_iter):
        # get loss and w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        if (iter % 10) == 0:
            print("Current iteration={%s}, loss={%s}" % (iter, loss))
        # converge check
        losses.append(loss)
        if len(losses) > 1 and (np.abs(losses[-1] - losses[-2]) < threshold):
            break

    loss = calculate_loss(y, tx, w)
    print("loss={%s}" % loss)
    plt.plot(losses)
    return w, loss

In [ ]:
w = logistic_regression(y_class, x_train)

In [ ]:
w

In [ ]:
testing_x = np.array([[2.3, 3.2], [1.0, 0.1], [1.4, 2.3]])
testing_y = np.array([0.1, 0.3, 0.5])
testing_y = (testing_y > 0.2) * 1.0
testing_init_w = np.array([0.5, 1.0])
testing_res_w, testing_res_loss = logistic_regression(testing_y, testing_x, max_iter=2, gamma=0.1, initial_w=testing_init_w, threshold=1.0)

expected_loss = 1.348358
expected_w = np.array([0.378561, 0.801131])

np.testing.assert_allclose(testing_res_loss, expected_loss, rtol=1e-4, atol=1e-8)
np.testing.assert_allclose(testing_res_w, expected_w, rtol=1e-4, atol=1e-8)
assert testing_res_loss.ndim == 0
assert testing_res_w.shape == expected_w.shape